In [11]:
%pwd

u'/home/ubuntu/courses/deeplearning1/nbs/my_work/lesson1'

In [12]:
import os
import glob

In [13]:
current_dir = os.getcwd()
data_dir = current_dir + "/data/redux/"

In [14]:
from utils import *
from vgg16 import Vgg16

%matplotlib inline

Using cuDNN version 5103 on context None
Mapped name None to device cuda: Tesla K80 (0000:00:1E.0)
Using Theano backend.


In [5]:
%cd data/redux/train/

/home/ubuntu/courses/deeplearning1/nbs/my_work/lesson1/data/redux/train


In [15]:
import numpy as np

In [8]:
files = glob("*.jpg")
shuf = np.random.permutation(files)
for i in range(2000):
    os.rename(shuf[i], data_dir + "/valid/" + shuf[i])

In [9]:
from shutil import copyfile

In [10]:
files = glob("*.jpg")
shuf = np.random.permutation(files)
for i in range(200):
    copyfile(shuf[i], data_dir + "/samp/train/" + shuf[i])

In [11]:
%cd ..

/home/ubuntu/courses/deeplearning1/nbs/my_work/lesson1/data/redux


In [12]:
%cd valid/

/home/ubuntu/courses/deeplearning1/nbs/my_work/lesson1/data/redux/valid


In [13]:
files = glob("*.jpg")
shuf = np.random.permutation(files)
for i in range(50):
    copyfile(shuf[i], data_dir + "/samp/valid/" + shuf[i])

In [14]:
%cd ..

/home/ubuntu/courses/deeplearning1/nbs/my_work/lesson1/data/redux


In [15]:
%cd train/

/home/ubuntu/courses/deeplearning1/nbs/my_work/lesson1/data/redux/train


In [16]:
%mkdir cats
%mkdir dogs

In [18]:
%mv cat.*.jpg cats/
%mv dog.*.jpg dogs/

In [19]:
%cd ..

/home/ubuntu/courses/deeplearning1/nbs/my_work/lesson1/data/redux


In [27]:
%cd train/

/home/ubuntu/courses/deeplearning1/nbs/my_work/lesson1/data/redux/samp/train


In [28]:
%mkdir cats
%mkdir dogs

In [29]:
%mv cat.*.jpg cats/
%mv dog.*.jpg dogs/

In [33]:
%mkdir cats

In [34]:
%mkdir dogs

In [35]:
%mv cat.*.jpg cats/
%mv dog.*.jpg dogs/

In [37]:
%cd valid/

/home/ubuntu/courses/deeplearning1/nbs/my_work/lesson1/data/redux/valid


In [39]:
%mkdir cats
%mkdir dogs

mkdir: cannot create directory ‘cats’: File exists


In [41]:
%mv cat.*.jpg cats/
%mv dog.*.jpg dogs/

In [42]:
%cd ..

/home/ubuntu/courses/deeplearning1/nbs/my_work/lesson1/data/redux


In [2]:
%cd data/redux/test/

/home/ubuntu/courses/deeplearning1/nbs/my_work/lesson1/data/redux/test


In [11]:
%mkdir unknown

In [12]:
%mv *.jpg unknown

In [14]:
%pwd

u'/home/ubuntu/courses/deeplearning1/nbs/my_work/lesson1/data/redux/test'

In [15]:
%cd ../

/home/ubuntu/courses/deeplearning1/nbs/my_work/lesson1/data/redux


In [16]:
%mkdir results

In [22]:
%cd data/redux/

/home/ubuntu/courses/deeplearning1/nbs/my_work/lesson1/data/redux


In [18]:
train_path = current_dir + "/train/"
val_path = current_dir + "/valid/"
test_path = current_dir + "/test/"
results_path = current_dir + "/results/"

In [26]:
batch_size = 64
num_epochs = 3

In [27]:
vgg = Vgg16()

In [35]:
train_batches = vgg.get_batches(train_path, batch_size=batch_size)
val_batches = vgg.get_batches(val_path, batch_size=batch_size)
vgg.finetune(train_batches)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [36]:
vgg.model.optimizer.lr = 0.01

In [38]:
for epoch in range(num_epochs):
    print "running epoch %d" % epoch
    vgg.fit(train_batches, val_batches, nb_epoch=1)
    latest_weights = "ft%d.h5" % epoch
print "completed %d epochs" % num_epochs

running epoch 0
Epoch 1/1
23000/23000 [==============================] - 480s - loss: 0.3281 - acc: 0.9716 - val_loss: 0.1792 - val_acc: 0.9855
running epoch 1
Epoch 1/1
23000/23000 [==============================] - 480s - loss: 0.3504 - acc: 0.9752 - val_loss: 0.2378 - val_acc: 0.9820
running epoch 2
Epoch 1/1
23000/23000 [==============================] - 480s - loss: 0.3290 - acc: 0.9772 - val_loss: 0.1573 - val_acc: 0.9885
completed 3 epochs


In [39]:
vgg.model.save_weights(results_path + latest_weights)

In [40]:
batches, probs = vgg.test(test_path, batch_size = batch_size)

Found 12500 images belonging to 1 classes.


In [41]:
dogs = probs[:,1]
filenames = batches.filenames

In [2]:
import numpy as np

In [44]:
save_array(results_path + "probs_dog.dat", dogs)
save_array(results_path + "filenames.dat", filenames)
save_array(results_path + "probs.dat", probs)

In [45]:
id = np.array([int(f[8:f.find(".")]) for f in filenames])

In [24]:
sub = np.stack([id,dogs], axis = 1)

NameError: name 'dogs' is not defined

In [47]:
sub[:3]

array([[  2.5080e+03,   9.2878e-42],
       [  9.4870e+03,   0.0000e+00],
       [  6.5900e+03,   1.0000e+00]])

In [48]:
%pwd

u'/home/ubuntu/courses/deeplearning1/nbs/my_work/lesson1/data/redux'

In [51]:
submission_filename = "submission_1.csv"
np.savetxt(submission_filename, sub, fmt = "%d,%.5f", header="id,label")

In [21]:
isdog = load_array("results/probs_dog.dat")
filenames = load_array("results/filenames.dat")

In [22]:
isdog = isdog.clip(min=0.02, max=0.98)

In [23]:
ids = np.array([int(f[8:f.find(".")]) for f in filenames])

In [25]:
sub1 = np.stack([ids, isdog], axis = 1)

In [26]:
sub1[:3]

array([[  2.5080e+03,   2.0000e-02],
       [  9.4870e+03,   2.0000e-02],
       [  6.5900e+03,   9.8000e-01]])

In [27]:
??np.savetxt

In [28]:
np.savetxt("submission_2.csv", sub1, delimiter=",", fmt = "%d,%.5f", header="id,label")